In [1]:
%pylab inline
%load_ext rpy2.ipython

# for managing data
import pandas as pd
import networkx as nx
import json
from collections import Counter
import itertools
import copy
from datetime import datetime

from lifelines import CoxTimeVaryingFitter
from sklearn.decomposition import PCA

Populating the interactive namespace from numpy and matplotlib


In [41]:
results = []
hazard_table_collector = []

## July 10 and 18

- Game split into two sessions
- Floating filler clues
- Don't remember if survey asked for a text description or a dropdown menu.

In [3]:
# Load games information
export_dir = "/Users/jameshoughton/Google Drive/MIT PhD/Factionalism_Research/Experiment/empirica/detective-game/design/results/Empirica Data - 2019-07-18 19-27-52/"
players = []
with open(export_dir + 'players.json', 'r') as f:
    for line in f:
        players.append(json.loads(line))

games = []
with open(export_dir + 'games.json', 'r') as f:
    for line in f:
        games.append(json.loads(line))
        
treatments = []
with open(export_dir + 'treatments.json', 'r') as f:
    for line in f:
        treatments.append(json.loads(line))

experiment_filepath = "/Users/jameshoughton/Google Drive/MIT PhD/Factionalism_Research/Experiment/empirica/detective-game/private/exp_20_3_4_13_10_220190708_172010.json"
with open(experiment_filepath, 'r') as f:
    experiment = json.load(f)
    

# match games, players, experiment data
experiment_games = []
for game in games:
    game['players'] = {pl['_id']:pl for pl in players if pl['_id'] in game['playerIds']}
    treatment = [t for t in treatments if t["_id"] == game['treatmentId']][0]
    game['gameSetupId'] = treatment['name']
    
    if game['gameSetupId'] in experiment['games'].keys():
        game['gameData'] = experiment['games'][game['gameSetupId']]
        experiment_games.append(game)
        print('%s loaded' %game['gameSetupId'])
    else:
        print('%s not in experiment' % game['gameSetupId'])
        
        
res = {'game': '0718'}

game_0718_control = experiment_games[-1]
game = game_0718_control

# Define clues used in polarization analysis
# final clues used in analysis are connections between hub nodes (1,2) and rim nodes (3-13)
c_spokes = ['clue_1_3', 'clue_1_4', 'clue_1_5', 'clue_1_6', 'clue_1_7', 
            'clue_1_8', 'clue_1_9', 'clue_1_10','clue_1_11', 'clue_1_12', 'clue_1_13',
            'clue_2_3', 'clue_2_4', 'clue_2_5', 'clue_2_6', 'clue_2_7',
            'clue_2_8', 'clue_2_9', 'clue_2_10', 'clue_2_11', 'clue_2_12', 'clue_2_13']

c_pos = game['players'].keys()
# Form final notebook states into a dataframe
final_adoptions = pd.DataFrame(data=0, index=c_pos, columns=c_spokes)
for p, k in game['players'].items():
    for clue_id in k['data.notebooks']['promising_leads']['clueIDs']:
        final_adoptions.loc[p, clue_id] = 1
        
# final state PC1
pca = PCA(n_components=1)
pca.fit(final_adoptions.loc[c_pos, c_spokes])  
res['c_spoke_pc1_variance'] = pca.explained_variance_ratio_[0]

# final state similarity percentiles
res['c_spoke_5th_similarity'], res['c_spoke_95th_similarity'] = np.percentile(
        final_adoptions.loc[c_pos, c_spokes].T.corr().mask(
            np.tri(20, 20, 0, dtype='bool')).stack(), [5, 95])


game_0718_tmt = experiment_games[-2]
game = game_0718_tmt

# Define clues used in polarization analysis
# final clues used in analysis are connections between hub nodes (1,2) and rim nodes (3-13)
t_spokes = ['clue_1_3', 'clue_1_4', 'clue_1_5', 'clue_1_6', 'clue_1_7', 
            'clue_1_8', 'clue_1_9', 'clue_1_10','clue_1_11', 'clue_1_12', 'clue_1_13',
            'clue_2_3', 'clue_2_4', 'clue_2_5', 'clue_2_6', 'clue_2_7',
            'clue_2_8', 'clue_2_9', 'clue_2_10', 'clue_2_11', 'clue_2_12', 'clue_2_13']

t_pos = game['players'].keys()

# Form initial notebook states into a dataframe
initial_adoptions = pd.DataFrame(data=0, index=t_pos, columns=t_spokes)
for p, k in game['players'].items():
    for clue_id in k['data.initialState']['promising_leads']['clueIDs']:
        initial_adoptions.loc[p, clue_id] = 1

# initial-state PC1
pca = PCA(n_components=1)
pca.fit(initial_adoptions.loc[t_pos, t_spokes])  
res['t_spoke_pc1_initial_variance'] = pca.explained_variance_ratio_


# Form final notebook states into a dataframe
final_adoptions = pd.DataFrame(data=0, index=t_pos, columns=t_spokes)
for p, k in game['players'].items():
    for clue_id in k['data.notebooks']['promising_leads']['clueIDs']:
        final_adoptions.loc[p, clue_id] = 1
        
# final state PC1
pca = PCA(n_components=1)
pca.fit(final_adoptions.loc[t_pos, t_spokes])  
res['t_spoke_pc1_variance'] = pca.explained_variance_ratio_[0]


res['t_spoke_5th_similarity'], res['t_spoke_95th_similarity'] = np.percentile(
    final_adoptions.loc[t_pos, t_spokes].T.corr().mask(
        np.tri(20, 20, 0, dtype='bool')).stack(), [5, 95])

#results.append(res)     
        

panel_0_treatment_0.00_exp_4_3_4_17_10_220190508_100624 not in experiment
panel_0_treatment_0.00_exp_20_3_4_13_10_220190515_104508 not in experiment
panel_0_treatment_0.00_exp_20_3_4_13_10_220190515_104508 not in experiment
panel_0_treatment_1.00_exp_4_3_4_17_10_220190508_100624 not in experiment
panel_0_treatment_1.00_exp_20_3_4_13_10_220190521_101219 not in experiment
panel_0_treatment_0.00_exp_20_3_4_13_10_220190708_172010 loaded
panel_0_treatment_1.00_exp_20_3_4_13_10_220190708_172010 loaded


# July 26

- matched pair
- floating (non-spur) dummy clues

In [100]:
export_dir = "/Users/jameshoughton/Google Drive/MIT PhD/Factionalism_Research/Experiment/empirica/detective-game/design/results/Empirica Data - 2019-07-26 18-24-03/"
players = []
with open(export_dir + 'players.json', 'r') as f:
    for line in f:
        players.append(json.loads(line))

games = []
with open(export_dir + 'games.json', 'r') as f:
    for line in f:
        games.append(json.loads(line))
        
treatments = []
with open(export_dir + 'treatments.json', 'r') as f:
    for line in f:
        treatments.append(json.loads(line))

experiment_filepath = "/Users/jameshoughton/Google Drive/MIT PhD/Factionalism_Research/Experiment/empirica/detective-game/design/results/exp_design3_matched_20190726_115504.json"
with open(experiment_filepath, 'r') as f:
    experiment = json.load(f)
    

# match games, players, experiment data
experiment_games = []
for game in games:
    game['players'] = {pl['_id']:pl for pl in players if pl['_id'] in game['playerIds']}
    treatment = [t for t in treatments if t["_id"] == game['treatmentId']][0]
    game['gameSetupId'] = treatment['name']
    
    if game['gameSetupId'] in experiment['games'].keys():
        game['gameData'] = experiment['games'][game['gameSetupId']]
        experiment_games.append(game)
        print('%s loaded' %game['gameSetupId'])
    else:
        print('%s not in experiment' % game['gameSetupId'])
        
game_0726 = experiment_games[-1]

game = game_0726

# Define clues used in polarization analysis
# final clues used in analysis are connections between hub nodes (1,2) and rim nodes (3-13)
t_spokes = ['tclue_1_3', 'tclue_1_4', 'tclue_1_5', 'tclue_1_6', 'tclue_1_7', 
            'tclue_1_8', 'tclue_1_9', 'tclue_1_10','tclue_1_11', 'tclue_1_12', 'tclue_1_13',
            'tclue_2_3', 'tclue_2_4', 'tclue_2_5', 'tclue_2_6', 'tclue_2_7',
            'tclue_2_8', 'tclue_2_9', 'tclue_2_10', 'tclue_2_11', 'tclue_2_12', 'tclue_2_13']

c_spokes = ['cclue_1_3', 'cclue_1_4', 'cclue_1_5', 'cclue_1_6', 'cclue_1_7', 
            'cclue_1_8', 'cclue_1_9', 'cclue_1_10','cclue_1_11', 'cclue_1_12', 'cclue_1_13',
            'cclue_2_3', 'cclue_2_4', 'cclue_2_5', 'cclue_2_6', 'cclue_2_7',
            'cclue_2_8', 'cclue_2_9', 'cclue_2_10', 'cclue_2_11', 'cclue_2_12', 'cclue_2_13']

t_pos = ['t%i'%i for i in range(20)]
c_pos = ['c%i'%i for i in range(20)]
pos = t_pos + c_pos

# Form initial notebook states into a dataframe
initial_adoptions = pd.DataFrame(data=0, index=t_pos+c_pos, columns=t_spokes+c_spokes)
for p, k in game['players'].items():
    for clue_id in k['data.initialState']['promising_leads']['clueIDs']:
        initial_adoptions.loc[k['data.position'], clue_id] = 1

# initial-state PC1
pca = PCA(n_components=1)
pca.fit(initial_adoptions.loc[t_pos, t_spokes])  
res['t_spoke_pc1_initial_variance'] = pca.explained_variance_ratio_

# Form final notebook states into a dataframe
final_adoptions = pd.DataFrame(data=0, index=t_pos+c_pos, columns=t_spokes+c_spokes)
for p, k in game['players'].items():
    for clue_id in k['data.notebooks']['promising_leads']['clueIDs']:
        final_adoptions.loc[k['data.position'], clue_id] = 1

res = {'game': '0726'}
# final state PC1
pca = PCA(n_components=1)
pca.fit(final_adoptions.loc[c_pos, c_spokes])  
res['c_spoke_pc1_variance'] = pca.explained_variance_ratio_[0]

pca = PCA(n_components=1)
pca.fit(final_adoptions.loc[t_pos, t_spokes])  
res['t_spoke_pc1_variance'] = pca.explained_variance_ratio_[0]

# final state similarity percentiles
res['c_spoke_5th_similarity'], res['c_spoke_95th_similarity'] = np.percentile(
        final_adoptions.loc[c_pos, c_spokes].T.corr().mask(
            np.tri(20, 20, 0, dtype='bool')).stack(), [5, 95])


res['t_spoke_5th_similarity'], res['t_spoke_95th_similarity'] = np.percentile(
    final_adoptions.loc[t_pos, t_spokes].T.corr().mask(
        np.tri(20, 20, 0, dtype='bool')).stack(), [5, 95])

results.append(res)

panel_1_matched_pair_exp_design3_matched_20190723_115409 not in experiment
panel_0_matched_pair_exp_design3_matched_20190723_115409 not in experiment
panel_0_matched_pair_exp_design3_matched_20190723_115409 not in experiment
panel_0_matched_pair_exp_design3_matched_20190726_115504 loaded


In [24]:
t_spokes = ['tclue_1_3', 'tclue_1_4', 'tclue_1_5', 'tclue_1_6', 'tclue_1_7', 
            'tclue_1_8', 'tclue_1_9', 'tclue_1_10','tclue_1_11', 'tclue_1_12', 'tclue_1_13',
            'tclue_2_3', 'tclue_2_4', 'tclue_2_5', 'tclue_2_6', 'tclue_2_7',
            'tclue_2_8', 'tclue_2_9', 'tclue_2_10', 'tclue_2_11', 'tclue_2_12', 'tclue_2_13']

c_spokes = ['cclue_1_3', 'cclue_1_4', 'cclue_1_5', 'cclue_1_6', 'cclue_1_7', 
            'cclue_1_8', 'cclue_1_9', 'cclue_1_10','cclue_1_11', 'cclue_1_12', 'cclue_1_13',
            'cclue_2_3', 'cclue_2_4', 'cclue_2_5', 'cclue_2_6', 'cclue_2_7',
            'cclue_2_8', 'cclue_2_9', 'cclue_2_10', 'cclue_2_11', 'cclue_2_12', 'cclue_2_13']


def who_has_what(g):
    holds = []
    for n in g:
        pos = g.node[n]['pos']
        for cl in t_spokes+c_spokes:
            edge = game['data.clues'][cl]['nodes']
            holds.append((pos, cl, g.node[n]['M'].has_edge(*edge)))

    holds_df = pd.DataFrame(holds, columns=['pos', 'clueId', 'present']).pivot(index='pos', columns='clueId', values='present')
    #holds_df.reset_index(inplace=True, drop=True)
    return holds_df

import copy
collector = []
pca = PCA(n_components=1)

# for (player_id, g, event) in retrace(game_1127):
#     adoptions = who_has_what(g)
#     pca.fit(adoptions.loc[t_pos, t_spokes])
#     res = {'t':event['t']/60,
#            'treatment': 1,
#            'pc1':pca.explained_variance_ratio_[0],
#            'game': 1127
#           }  
#     collector.append(res)

#     pca.fit(adoptions.loc[c_pos, c_spokes])
#     res = {'t': event['t']/60,
#            'treatment': 0,
#            'pc1': pca.explained_variance_ratio_[0],
#            'game': 1127
#           }  
#     collector.append(res)
    
# for (player_id, g, event) in retrace(game_1001):
#     adoptions = who_has_what(g)
#     pca.fit(adoptions.loc[t_pos, t_spokes])
#     res = {'t':event['t']/60,
#            'treatment': 1,
#            'pc1':pca.explained_variance_ratio_[0],
#            'game': 1001
#           }  
#     collector.append(res)

#     pca.fit(adoptions.loc[c_pos, c_spokes])
#     res = {'t': event['t']/60,
#            'treatment': 0,
#            'pc1': pca.explained_variance_ratio_[0],
#            'game': 1001
#           }  
#     collector.append(res)
        


# October 1

- Matched pair
- Poorly constructed spur clues
- Slider survey - sliders have default values, and force you to move away from center in order to submit a result...

In [42]:

export_dir = "/Users/jameshoughton/Google Drive/MIT PhD/Factionalism_Research/Experiment/empirica/detective-game/design/results/Empirica Data - 2019-10-01 18-28-44/"
players = []
with open(export_dir + 'players.json', 'r') as f:
    for line in f:
        players.append(json.loads(line))

games = []
with open(export_dir + 'games.json', 'r') as f:
    for line in f:
        games.append(json.loads(line))
        
treatments = []
with open(export_dir + 'treatments.json', 'r') as f:
    for line in f:
        treatments.append(json.loads(line))

experiment_filepath = "/Users/jameshoughton/Google Drive/MIT PhD/Factionalism_Research/Experiment/empirica/detective-game/design/results/exp_design4_matched_20190925_121500.json"
with open(experiment_filepath, 'r') as f:
    experiment = json.load(f)
    

# match games, players, experiment data
experiment_games = []
for game in games:
    game['players'] = {pl['_id']:pl for pl in players if pl['_id'] in game['playerIds']}
    treatment = [t for t in treatments if t["_id"] == game['treatmentId']][0]
    game['gameSetupId'] = treatment['name']
    
    if game['gameSetupId'] in experiment['games'].keys():
        game['gameData'] = experiment['games'][game['gameSetupId']]
        game['data.clues'] = experiment['games'][game['gameSetupId']]['clues']
        experiment_games.append(game)
        print('%s loaded' %game['gameSetupId'])
    else:
        print('%s not in experiment' % game['gameSetupId'])
        
game_1001 = experiment_games[-1]      


game = game_1001
res = {'game': '1001'}

# Define clues used in polarization analysis
# final clues used in analysis are connections between hub nodes (1,2) and rim nodes (3-13)
t_spokes = ['tclue_1_3', 'tclue_1_4', 'tclue_1_5', 'tclue_1_6', 'tclue_1_7', 
            'tclue_1_8', 'tclue_1_9', 'tclue_1_10','tclue_1_11', 'tclue_1_12', 'tclue_1_13',
            'tclue_2_3', 'tclue_2_4', 'tclue_2_5', 'tclue_2_6', 'tclue_2_7',
            'tclue_2_8', 'tclue_2_9', 'tclue_2_10', 'tclue_2_11', 'tclue_2_12', 'tclue_2_13']

c_spokes = ['cclue_1_3', 'cclue_1_4', 'cclue_1_5', 'cclue_1_6', 'cclue_1_7', 
            'cclue_1_8', 'cclue_1_9', 'cclue_1_10','cclue_1_11', 'cclue_1_12', 'cclue_1_13',
            'cclue_2_3', 'cclue_2_4', 'cclue_2_5', 'cclue_2_6', 'cclue_2_7',
            'cclue_2_8', 'cclue_2_9', 'cclue_2_10', 'cclue_2_11', 'cclue_2_12', 'cclue_2_13']

t_pos = ['t%i'%i for i in range(20)]
c_pos = ['c%i'%i for i in range(20)]
pos = t_pos + c_pos

# Form initial notebook states into a dataframe
initial_adoptions = pd.DataFrame(data=0, index=t_pos+c_pos, columns=t_spokes+c_spokes)
for p, k in game['players'].items():
    for clue_id in k['data.initialState']['promising_leads']['clueIDs']:
        initial_adoptions.loc[k['data.position'], clue_id] = 1

# initial-state PC1
pca = PCA(n_components=1)
pca.fit(initial_adoptions.loc[t_pos, t_spokes])  
res['t_spoke_pc1_initial_variance'] = pca.explained_variance_ratio_[0]

# Form final notebook states into a dataframe
final_adoptions = pd.DataFrame(data=0, index=t_pos+c_pos, columns=t_spokes+c_spokes)
for p, k in game['players'].items():
    for clue_id in k['data.notebooks']['promising_leads']['clueIDs']:
        final_adoptions.loc[k['data.position'], clue_id] = 1

# final state PC1
pca = PCA(n_components=1)
pca.fit(final_adoptions.loc[c_pos, c_spokes])  
res['c_spoke_pc1_variance'] = pca.explained_variance_ratio_[0]

pca = PCA(n_components=1)
pca.fit(final_adoptions.loc[t_pos, t_spokes])  
res['t_spoke_pc1_variance'] = pca.explained_variance_ratio_[0]

# final state similarity percentiles
res['c_spoke_5th_similarity'], res['c_spoke_95th_similarity'] = np.percentile(
        final_adoptions.loc[c_pos, c_spokes].T.corr().mask(
            np.tri(20, 20, 0, dtype='bool')).stack(), [5, 95])


res['t_spoke_5th_similarity'], res['t_spoke_95th_similarity'] = np.percentile(
    final_adoptions.loc[t_pos, t_spokes].T.corr().mask(
        np.tri(20, 20, 0, dtype='bool')).stack(), [5, 95])

results.append(res)

# for (player_id, g, event) in retrace(game_1001):
#     adoptions = who_has_what(g)
#     pca.fit(adoptions.loc[t_pos, t_spokes])
#     res = {'t':event['t']/60,
#            'treatment': 1,
#            'pc1':pca.explained_variance_ratio_[0],
#            'game': 1001
#           }  
#     collector.append(res)

#     pca.fit(adoptions.loc[c_pos, c_spokes])
#     res = {'t': event['t']/60,
#            'treatment': 0,
#            'pc1': pca.explained_variance_ratio_[0],
#            'game': 1001
#           }  
#     collector.append(res)



panel_1_matched_pair_exp_design3_matched_20190723_115409 not in experiment
panel_0_matched_pair_exp_design3_matched_20190723_115409 not in experiment
panel_0_matched_pair_exp_design3_matched_20190723_115409 not in experiment
panel_0_matched_pair_exp_design3_matched_20190726_115504 not in experiment
panel_0_matched_pair_exp_design4_matched_20190925_121500 loaded
panel_0_matched_pair_exp_design4_matched_20190925_121500 loaded


# november 27

- with Abdullah
- Matched pair 
- properly tested spurs
- bad time logging - turns out I'd been relying on client side clocks to capture events, and so there is no guarantee of synchronization between the different players!
- slider survey
- problem with slider labels

In [43]:
# load game

export_dir = "/Users/jameshoughton/Google Drive/MIT PhD/Factionalism_Research/Experiment/empirica/detective-game/design/results/Empirica Data - 2019-11-27 18-38-43/"
players = []
with open(export_dir + 'players.json', 'r') as f:
    for line in f:
        players.append(json.loads(line))

games = []
with open(export_dir + 'games.json', 'r') as f:
    for line in f:
        games.append(json.loads(line))
        
treatments = []
with open(export_dir + 'treatments.json', 'r') as f:
    for line in f:
        treatments.append(json.loads(line))


# match games, players, and treatments
loaded_games = []
for game in games:
    game['players'] = {pl['_id']:pl for pl in players if pl['_id'] in game['playerIds']}
    treatment = [t for t in treatments if t["_id"] == game['treatmentId']][0]
    game['gameSetupId'] = treatment['name']
        
    
    loaded_games.append(game)
    
for i, game in enumerate(loaded_games):
    print(i, game['createdAt'])
    
# select which games to process
select = [0]
process_games = [loaded_games[i] for i in select]

game_1127 = process_games[-1] 


game = game_1127
# Define clues used in polarization analysis
# final clues used in analysis are connections between hub nodes (1,2) and rim nodes (3-13)
t_spokes = ['tclue_1_3', 'tclue_1_4', 'tclue_1_5', 'tclue_1_6', 'tclue_1_7', 
            'tclue_1_8', 'tclue_1_9', 'tclue_1_10','tclue_1_11', 'tclue_1_12', 'tclue_1_13',
            'tclue_2_3', 'tclue_2_4', 'tclue_2_5', 'tclue_2_6', 'tclue_2_7',
            'tclue_2_8', 'tclue_2_9', 'tclue_2_10', 'tclue_2_11', 'tclue_2_12', 'tclue_2_13']

c_spokes = ['cclue_1_3', 'cclue_1_4', 'cclue_1_5', 'cclue_1_6', 'cclue_1_7', 
            'cclue_1_8', 'cclue_1_9', 'cclue_1_10','cclue_1_11', 'cclue_1_12', 'cclue_1_13',
            'cclue_2_3', 'cclue_2_4', 'cclue_2_5', 'cclue_2_6', 'cclue_2_7',
            'cclue_2_8', 'cclue_2_9', 'cclue_2_10', 'cclue_2_11', 'cclue_2_12', 'cclue_2_13']

t_pos = ['t%i'%i for i in range(20)]
c_pos = ['c%i'%i for i in range(20)]
pos = t_pos + c_pos

# Form final notebook states into a dataframe
final_adoptions = pd.DataFrame(data=0, index=t_pos+c_pos, columns=t_spokes+c_spokes)
for p, k in game['players'].items():
    for clue_id in k['data.notebooks']['promising_leads']['clueIDs']:
        final_adoptions.loc[k['data.position'], clue_id] = 1

res = {'game': '1127'}

# Form initial notebook states into a dataframe
initial_adoptions = pd.DataFrame(data=0, index=t_pos+c_pos, columns=t_spokes+c_spokes)
for p, k in game['players'].items():
    for clue_id in k['data.initialState']['promising_leads']['clueIDs']:
        initial_adoptions.loc[k['data.position'], clue_id] = 1

# initial-state PC1
pca = PCA(n_components=1)
pca.fit(initial_adoptions.loc[t_pos, t_spokes])  
res['t_spoke_pc1_initial_variance'] = pca.explained_variance_ratio_[0]

# final state PC1
pca = PCA(n_components=1)
pca.fit(final_adoptions.loc[c_pos, c_spokes])  
res['c_spoke_pc1_variance'] = pca.explained_variance_ratio_[0]


pca = PCA(n_components=1)
pca.fit(final_adoptions.loc[t_pos, t_spokes])  
res['t_spoke_pc1_variance'] = pca.explained_variance_ratio_[0]

# final state similarity percentiles
res['c_spoke_5th_similarity'], res['c_spoke_95th_similarity'] = np.percentile(
        final_adoptions.loc[c_pos, c_spokes].T.corr().mask(
            np.tri(20, 20, 0, dtype='bool')).stack(), [5, 95])


res['t_spoke_5th_similarity'], res['t_spoke_95th_similarity'] = np.percentile(
    final_adoptions.loc[t_pos, t_spokes].T.corr().mask(
        np.tri(20, 20, 0, dtype='bool')).stack(), [5, 95])

results.append(res)

# def retrace(game):
#     """
#     Uses the game log and starting conditions to recreate the state of the
#     game at every change event.
    
#     Returns a generator yielding (player_id, g, event) at each event in the game,

#     *player_id* is the player logging the event,
#     *g* is the state of the game following the event,
#     *event* is what is logged, timestamp modified to be in seconds since game start

#     Does not return an action if the only change is a list reordering.
#     """
#     #game_data = game['gameData']
#     clues = game['data.clues']
    
#     # create trace social network
#     edge_list = []
#     for player_id, player_data in game['players'].items():
#         for alter_id in player_data['data.alterIDs']:
#             edge_list.append([player_id, alter_id])
#     g = nx.from_edgelist(edge_list)

#     # give trace players starting information
#     nx.set_node_attributes(
#         g,
#         name='pos',  # position in the social network
#         values={a: game['players'][a]['data.position'] for a in g}
#     )

#     nx.set_node_attributes(
#         g,
#         name='M',  # M for mind/memory
#         values={a: nx.from_edgelist([
#             clues[bf]['nodes'] for bf in
#             #game['players'][a]['data.intialState']['promising_leads']['clueIDs']
#             game['players'][a]['data.initialState']['promising_leads']['clueIDs']
#         ]) for a in g}
#     )

#     nx.set_node_attributes(
#         g,
#         name='F',  # F for forgetory
#         values={i: nx.Graph() for i in g}
#     )

#     # yield the initial state of the experiment
#     event = {'at': game['createdAt'],
#              't': 0}
#     yield (None, g, event)

#     # collect the log of the entire experiment by combining all player logs
#     game_log = pd.DataFrame()
#     for player_id, player_data in game['players'].items():
#         player_log = pd.DataFrame(player_data['data.log'])
#         player_log['player'] = player_id
#         game_log = game_log.append(player_log)
#     game_log.sort_values('at', inplace=True)

#     # trace game
#     t_start = datetime.strptime(game['createdAt'], '%Y-%m-%dT%H:%M:%S.%fZ')

#     for _, event in game_log[game_log['event'] == "drop"].iterrows():
#         player_id = event["player"]
#         source = event['data']['source']
#         dest = event['data']['dest']
#         if 'clue' in event['data']:
#             if event['data']['clue'] != None:
#                 edge = clues[event['data']['clue']]['nodes']
#             else: # catch incomplete record
#                 print('Missing clueID for player %s from source %s at time %s' % (player_id, source, event['at']))
#         else:
#             print('player %s is missing a clue' % player_id)
#             continue
#         M = g.node[player_id]['M']
#         F = g.node[player_id]['F']
#         update = False

#         if source == "promising_leads":
#             assert g.node[event['player']]['M'].has_edge(*edge) # check that clue is still in memory
#             if dest == "dead_ends":
#                 M.remove_edge(*edge)
#                 F.add_edge(*edge)
#                 update = True

#         elif source == "dead_ends":
#             assert g.node[event['player']]['F'].has_edge(*edge) # check that clue is still in forgettory
#             if dest == "promising_leads":
#                 F.remove_edge(*edge)
#                 M.add_edge(*edge)
#                 update = True

#         else:
#             assert source in game['playerIds']  # check that source is another player
#             if not g.node[source]['M'].has_edge(*edge):  # check that clue is in source
#                 # this can fail if the exposer removes the clue while the exposed is dragging it.
#                 # turns out not to be a big deal
#                 print("%s no longer in source %s" % (str(edge), str(source)))
#             if dest == "promising_leads":
#                 M.add_edge(*edge)
#                 if F.has_edge(*edge):
#                     F.remove_edge(*edge)
#                 update = True
#             elif dest == "dead_ends":
#                 F.add_edge(*edge)
#                 if M.has_edge(*edge):
#                     M.remove_edge(*edge)
#                 update = True
#             assert not (F.has_edge(*edge) and  # not in both memory and forgetery
#                         M.has_edge(*edge))

#         if update:
#             t_current = datetime.strptime(event['at'], '%Y-%m-%dT%H:%M:%S.%fZ')
#             event['t'] = (t_current - t_start).total_seconds()
#             yield (player_id, g, event)

#     # double check the final state at the end of the generator
#     for player_id in g:
#         leads = game['players'][player_id]['data.notebooks']['promising_leads']['clueIDs']
#         should_have = set([tuple(sorted(clues[clue]['nodes'])) for clue in leads if clue != None])
#         has = set([tuple(sorted(edge)) for edge in g.node[player_id]['M'].edges()])
#         assert should_have == has

#         deads = game['players'][player_id]['data.notebooks']['dead_ends']['clueIDs']
#         should_have = set([tuple(sorted(clues[clue]['nodes'])) for clue in deads if clue != None])
#         has = set([tuple(sorted(edge)) for edge in g.node[player_id]['F'].edges()])
#         assert should_have == has
       
    
    
# def who_has_what(g):
#     holds = []
#     for n in g:
#         pos = g.node[n]['pos']
#         for cl in t_spokes+c_spokes:
#             edge = game['data.clues'][cl]['nodes']
#             holds.append((pos, cl, g.node[n]['M'].has_edge(*edge)))

#     holds_df = pd.DataFrame(holds, columns=['pos', 'clueId', 'present']).pivot(index='pos', columns='clueId', values='present')
#     #holds_df.reset_index(inplace=True, drop=True)
#     return holds_df    
    

# for (player_id, g, event) in retrace(game_1127):
#     adoptions = who_has_what(g)
#     pca.fit(adoptions.loc[t_pos, t_spokes])
#     res = {'t':event['t']/60,
#            'treatment': 1,
#            'pc1':pca.explained_variance_ratio_[0],
#            'game': 1127
#           }  
#     collector.append(res)

#     pca.fit(adoptions.loc[c_pos, c_spokes])
#     res = {'t': event['t']/60,
#            'treatment': 0,
#            'pc1': pca.explained_variance_ratio_[0],
#            'game': 1127
#           }  
#     collector.append(res)



0 2019-11-27T18:20:38.972Z


# December 11th

- Matched Pair
- Good spur clues
- Fixed timing of logs
- Slider survey



In [44]:
# load game

export_dir = "/Users/jameshoughton/Google Drive/MIT PhD/Factionalism_Research/Experiment/empirica/detective-game/design/results/Empirica Data - 2019-12-11 18-38-30/"
players = []
with open(export_dir + 'players.json', 'r') as f:
    for line in f:
        players.append(json.loads(line))

games = []
with open(export_dir + 'games.json', 'r') as f:
    for line in f:
        games.append(json.loads(line))
        
treatments = []
with open(export_dir + 'treatments.json', 'r') as f:
    for line in f:
        treatments.append(json.loads(line))

logs = []
with open(export_dir + 'player-logs.json', 'r') as f:
    for line in f:
        entry = json.loads(line)
        entry['data'] = json.loads(entry['jsonData'])
        logs.append(entry)

# match games, players, treatments, and log info
loaded_games = []
for game in games:
    game['players'] = {pl['_id']:pl for pl in players if pl['_id'] in game['playerIds']}
    treatment = [t for t in treatments if t["_id"] == game['treatmentId']][0]
    game['gameSetupId'] = treatment['name']
    game['log'] = [l for l in logs if l['gameId'] == game['_id']]
    
    loaded_games.append(game)
    
for i, game in enumerate(loaded_games):
    print(i, game['createdAt'])
    
# select which games to process
game = loaded_games[-1] 

# Define clues used in polarization analysis
# final clues used in analysis are connections between hub nodes (1,2) and rim nodes (3-13)
t_spokes = ['tclue_1_3', 'tclue_1_4', 'tclue_1_5', 'tclue_1_6', 'tclue_1_7', 
            'tclue_1_8', 'tclue_1_9', 'tclue_1_10','tclue_1_11', 'tclue_1_12', 'tclue_1_13',
            'tclue_2_3', 'tclue_2_4', 'tclue_2_5', 'tclue_2_6', 'tclue_2_7',
            'tclue_2_8', 'tclue_2_9', 'tclue_2_10', 'tclue_2_11', 'tclue_2_12', 'tclue_2_13']

c_spokes = ['cclue_1_3', 'cclue_1_4', 'cclue_1_5', 'cclue_1_6', 'cclue_1_7', 
            'cclue_1_8', 'cclue_1_9', 'cclue_1_10','cclue_1_11', 'cclue_1_12', 'cclue_1_13',
            'cclue_2_3', 'cclue_2_4', 'cclue_2_5', 'cclue_2_6', 'cclue_2_7',
            'cclue_2_8', 'cclue_2_9', 'cclue_2_10', 'cclue_2_11', 'cclue_2_12', 'cclue_2_13']

t_pos = ['t%i'%i for i in range(20)]
c_pos = ['c%i'%i for i in range(20)]
pos = t_pos + c_pos

# Form final notebook states into a dataframe
final_adoptions = pd.DataFrame(data=0, index=t_pos+c_pos, columns=t_spokes+c_spokes)
for p, k in game['players'].items():
    for clue_id in k['data.notebooks']['promising_leads']['clueIDs']:
        final_adoptions.loc[k['data.position'], clue_id] = 1

res = {'game': '1211'}

# Form initial notebook states into a dataframe
initial_adoptions = pd.DataFrame(data=0, index=t_pos+c_pos, columns=t_spokes+c_spokes)
for p, k in game['players'].items():
    for clue_id in k['data.initialState']['promising_leads']['clueIDs']:
        initial_adoptions.loc[k['data.position'], clue_id] = 1

# initial-state PC1
pca = PCA(n_components=1)
pca.fit(initial_adoptions.loc[t_pos, t_spokes])  
res['t_spoke_pc1_initial_variance'] = pca.explained_variance_ratio_[0]

# final state PC1
pca = PCA(n_components=1)
pca.fit(final_adoptions.loc[c_pos, c_spokes])  
res['c_spoke_pc1_variance'] = pca.explained_variance_ratio_[0]


pca = PCA(n_components=1)
pca.fit(final_adoptions.loc[t_pos, t_spokes])  
res['t_spoke_pc1_variance'] = pca.explained_variance_ratio_[0]

# final state similarity percentiles
res['c_spoke_5th_similarity'], res['c_spoke_95th_similarity'] = np.percentile(
        final_adoptions.loc[c_pos, c_spokes].T.corr().mask(
            np.tri(20, 20, 0, dtype='bool')).stack(), [5, 95])


res['t_spoke_5th_similarity'], res['t_spoke_95th_similarity'] = np.percentile(
    final_adoptions.loc[t_pos, t_spokes].T.corr().mask(
        np.tri(20, 20, 0, dtype='bool')).stack(), [5, 95])


# survey PC1
pca = PCA(n_components=1)
pca.fit(responses.loc[subset, assessments])  
sub_res['c_survey_pc1_variance'] = pca.explained_variance_ratio_

# survey similarity percentiles
sub_res['c_survey_5th_similarity'], sub_res['c_survey_95th_similarity'] = np.percentile(
    responses.loc[subset, assessments].T.corr().mask(
        np.tri(n_used, n_used, 0, dtype='bool')).stack(), [5, 95])

results.append(res)

assert False
## process timeseries parts
def retrace(game):
    """
    Uses the game log and starting conditions to recreate the state of the
    game at every change event.
    
    Returns a generator yielding (player_id, g, event) at each event in the game,

    *player_id* is the player logging the event,
    *g* is the state of the game following the event,
    *event* is what is logged, timestamp modified to be in seconds since game start

    Does not return an action if the only change is a list reordering.
    """
    #game_data = game['gameData']
    clues = game['data.clues']
    
    # create trace social network
    edge_list = []
    for player_id, player_data in game['players'].items():
        for alter_id in player_data['data.alterIDs']:
            edge_list.append([player_id, alter_id])
    g = nx.from_edgelist(edge_list)

    # give trace players starting information
    nx.set_node_attributes(
        g,
        name='pos',  # position in the social network
        values={a: game['players'][a]['data.position'] for a in g}
    )

    nx.set_node_attributes(
        g,
        name='M',  # M for mind/memory
        values={a: nx.from_edgelist([
            clues[bf]['nodes'] for bf in
            #game['players'][a]['data.intialState']['promising_leads']['clueIDs']
            game['players'][a]['data.initialState']['promising_leads']['clueIDs']
        ]) for a in g}
    )

    nx.set_node_attributes(
        g,
        name='F',  # F for forgetory
        values={i: nx.Graph() for i in g}
    )

    # yield the initial state of the experiment
    yield (None, g, 0)

    t_start = datetime.strptime(game['createdAt'], '%Y-%m-%dT%H:%M:%S.%fZ')

    game_log = pd.DataFrame(game['log'])
    game_log['logTime'] = game_log['createdAt'].apply(lambda s: datetime.strptime(s, '%Y-%m-%dT%H:%M:%S.%fZ'))
    game_log['t'] = (game_log['logTime'] - t_start).dt.total_seconds()
    game_log.sort_values('t', inplace=True)

    # trace game
    for _, row in game_log[game_log['name'] == "drop"].iterrows():
        player_id = row["playerId"]
        jsonData = json.loads(row['jsonData'])        
        source = jsonData['source']
        dest = jsonData['dest']

        if 'clue' in jsonData:
            if jsonData['clue'] != None:
                edge = clues[jsonData['clue']]['nodes']
            else: # catch incomplete record
                print('Missing clueID for player %s from source %s at time %s' % (player_id, source, row['t']))
        else:
            print('player %s is missing a clue' % player_id)
            continue
        M = g.node[player_id]['M']
        F = g.node[player_id]['F']
        update = False

        if source == "promising_leads":
            assert M.has_edge(*edge) # check that clue is still in memory
            if dest == "dead_ends":
                M.remove_edge(*edge)
                F.add_edge(*edge)
                update = True

        elif source == "dead_ends":
            assert F.has_edge(*edge) # check that clue is still in forgettory
            if dest == "promising_leads":
                F.remove_edge(*edge)
                M.add_edge(*edge)
                update = True

        else:
            assert source in game['playerIds']  # check that source is another player
            if not g.node[source]['M'].has_edge(*edge):  # check that clue is in source
                # this can fail if the exposer removes the clue while the exposed is dragging it.
                # turns out not to be a big deal
                # really we should only run this check on 'pickup' events, and then check that a 'drop' 
                # event follows
                print("%s no longer in source %s" % (str(edge), str(source)))
            if dest == "promising_leads":
                M.add_edge(*edge)
                if F.has_edge(*edge):
                    F.remove_edge(*edge)
                update = True
            elif dest == "dead_ends":
                F.add_edge(*edge)
                if M.has_edge(*edge):
                    M.remove_edge(*edge)
                update = True
            assert not (F.has_edge(*edge) and  # not in both memory and forgetery
                        M.has_edge(*edge))

        if update:
            yield (player_id, g, row['t'])

    # double check the final state at the end of the generator
    for player_id in g:
        leads = game['players'][player_id]['data.notebooks']['promising_leads']['clueIDs']
        should_have = set([tuple(sorted(clues[clue]['nodes'])) for clue in leads if clue != None])
        has = set([tuple(sorted(edge)) for edge in g.node[player_id]['M'].edges()])
        assert should_have == has

        deads = game['players'][player_id]['data.notebooks']['dead_ends']['clueIDs']
        should_have = set([tuple(sorted(clues[clue]['nodes'])) for clue in deads if clue != None])
        has = set([tuple(sorted(edge)) for edge in g.node[player_id]['F'].edges()])
        assert should_have == has
       

def hazard_factors(game, g, t):
    """
    Given the state of the game, what are the factors that lead to adoption?
    
    Returns a row for the hazard factors of all individuals adopting all beliefs 
    at the time of the event, so this gets big fast

    """

    rows = []
    players = game['players']
    hub_elements = {game['data.nodes']['CrimeScene_1'], game['data.nodes']['StolenObject_1']}
    clue_ids = list(game['data.clues'])
    spoke_clue_ids = t_spokes+c_spokes  # defined in the outside scope
    spoke_edges = [game['data.clues'][clue_id]['nodes'] for clue_id in spoke_clue_ids]
    
    for player_id in players:
        position = players[player_id]['data.position']
        neighbors = g.neighbors(player_id)
        M = copy.deepcopy(g.node[player_id]['M'])  # promising leads (memory)
        F = copy.deepcopy(g.node[player_id]['F'])  # dead ends (forgetory)
        
        neighbor_clues = {nb: [] for nb in neighbors}
        exposers = {clue_id: [] for clue_id in clue_ids}
        player_leads = []
        player_deads = []
        for clue_id in clue_ids:
            if clue_id[0] != position[0]: # only collect treatment clues for treatment players, vice versa
                continue 
            edge = game['data.clues'][clue_id]['nodes']
            if M.has_edge(*edge):
                player_leads.append(clue_id)
            if F.has_edge(*edge):
                player_deads.append(clue_id)
                
            for nb in g.neighbors(player_id):
                if g.node[nb]['M'].has_edge(*edge):
                    neighbor_clues[nb].append(clue_id)
                    exposers[clue_id].append(nb)
            
        fresh_candidates = set([cl for cl, exps in exposers.items() if len(exps) > 0]) - set(player_leads) - set(player_deads)    
        n_fresh_candidates = len(fresh_candidates)
        
        n_existing_beliefs = len(player_leads)
        
        
        # Removing the central clue means that we're only going to count logical interactions
        # that go via one of the other spoke clues. Under control conditions there should never 
        # be a logical interaction. I do this after counting the number of existing beliefs.
        if M.has_edge(*tuple(hub_elements)):
            M = copy.deepcopy(M)
            M.remove_edge(*tuple(hub_elements)) # ignore the clue linking the crime scene to stolen object

        
        for clue_id in spoke_clue_ids:
            if clue_id[0] != position[0]: # only collect treatment clues for treatment players, vice versa
                continue 
                
            edge = game['data.clues'][clue_id]['nodes']
            rim_node = (set(edge) - hub_elements).pop()
            
            row = {'game_player_clue_id': "%s_%s_%s" % (game['createdAt'], position, clue_id),
                   'game_player_id': "%s_%s" % (game['createdAt'], position),
                   'start': t,
                   'treatment': position.startswith('t'),
                   'have_belief': clue_id in player_leads,
                   'in_deads': 1.0*(clue_id in player_deads),  # cast as float for regression
                   'n_exposures': len(exposers[clue_id]),
                   'n_existing_beliefs':n_existing_beliefs,
                   'n_fresh_candidates': n_fresh_candidates,
                   'player_clues': player_leads,
                  }
            
            # number of times the rim element is referenced in the existing belief set
            if set(edge) == hub_elements:
                row['references'] = np.nan
            else:
                row['references'] = len(M.edges(rim_node))
                
                
            # count of paths player has connecting ends of clue
            path_list = nx.all_simple_paths(M, *edge, cutoff=2) if set(edge)<set(M.nodes()) else []
            path_counts = pd.Series(pd.Series([len(pth) - 1 for pth in path_list]).value_counts(),
                                    index=range(1, 5)).fillna(0)
            row['pl2'] = path_counts[2]

            row['exposers_clues'] = {nb: cls for nb, cls in neighbor_clues.items() if nb in exposers[clue_id]}
            
            rows.append(row)

    return rows


def drop_consecutive_duplicates(df, cols, sort_on):
    df = df.sort_values('start')
    return df.loc[any(df.shift(-1)[cols] != df[cols], axis=1).shift(+1).fillna('True').astype(bool)]

def add_stops(df, t_last):
    df['stop'] = df['start'].shift(-1)
    df.loc[df.index[-1], 'stop'] = t_last
    return df

def similarity_func(player_clues, alter_clues):
    # fraction of alter's total clues that player holds
    return len(set(player_clues) & set(alter_clues)) / len(alter_clues)


def process_adoption_group(group):
    """
    Takes the hazard table and creates a table that lifelines can use.
    1. Condenses multiple rows (by dropping duplicates)
    2. Treats start and end times
    3. Identifies adoption events
    """
    
    return_cols=[
        'treatment', 'game_player_clue_id', 'game_player_id', 'start', 'stop', 'adopt_event', 'forget_event',
        'n_exposures', 'n_fresh_candidates', 'have_belief',
        'pl2', 'references', 'n_existing_beliefs', 'in_deads',
        'max_similarity_to_exposer'
    ]
    
    #indexes at which exposures increase
    i_exposures = group.index[group['n_exposures'] > group.shift(+1)['n_exposures'].fillna(0)].tolist()
    if len(i_exposures) == 0:  # never exposed, throw out
        return pd.DataFrame()
    
    t_first_exposure = group[group['n_exposures']>0]['start'].min()
    if t_first_exposure < 1:  # exposed too early, throw out
         return pd.DataFrame()
    group['t_first_exposure'] = t_first_exposure
     
    # identify the exposer's clues at the time they expose you to the belief
    exposer_exposure_beliefs = {} # beliefs exposer held at the time of exposure
    for i in i_exposures:
        exposers_clues = group.loc[i]['exposers_clues']  # extract from array
        for exposer, clues in exposers_clues.items():
            if exposer not in exposer_exposure_beliefs: # new exposure
                exposer_exposure_beliefs[exposer] = clues
    
    # use the list of current exposers to figure out the max similarity to any exposer at the time of exposure
    max_similarity_to_exposer = []
    for i, row in group.iterrows():
        if len(row['exposers_clues']) == 0:
            max_similarity_to_exposer.append(-1000)  # this should be nan, but the drop duplicates can't handle that
        else:
            similarities = [similarity_func(row['player_clues'], exposer_exposure_beliefs[exposer])
                            for exposer in row['exposers_clues']]
            max_similarity_to_exposer.append(np.max(similarities))
    group['max_similarity_to_exposer'] = max_similarity_to_exposer
    
    # 'stops' for current (short period) rows
    t_final = datetime.strptime(game['finishedAt'], '%Y-%m-%dT%H:%M:%S.%fZ')
    t_start = datetime.strptime(game['createdAt'], '%Y-%m-%dT%H:%M:%S.%fZ')
    total_time = (t_final-t_start).total_seconds()
    group = add_stops(group, total_time) 
    
    fgroup = group[
        (group['stop'] < 450) & # throw out periods after the right censor
        (group['start'] > 1) # throw out before left censor
    ] 
    
    if len(fgroup) == 0: # kick out empty groups before we bother with the rest
        return pd.DataFrame() 
        
    
    fgroup = drop_consecutive_duplicates(
        df=fgroup, 
        cols=['n_exposures', 'max_similarity_to_exposer', 'have_belief', 
              'references', 'pl2', 'n_existing_beliefs', 'in_deads', 
              'n_fresh_candidates'], 
        sort_on='start'
    )
    fgroup = add_stops(fgroup, total_time) # update stops for longer periods
    
    fgroup['adopt_event'] = fgroup['have_belief'] < fgroup.shift(-1)['have_belief']
    fgroup.loc[fgroup.index[-1], 'adopt_event'] = False
    
    fgroup['forget_event'] = fgroup['have_belief'] > fgroup.shift(-1)['have_belief']
    fgroup.loc[fgroup.index[-1], 'forget_event'] = False
    
    return fgroup[return_cols]


hazard_factors_list = []
for (player_id, g, t) in retrace(game):
    hazard_factors_list += hazard_factors(game, g, t)
hazard_table = pd.DataFrame(hazard_factors_list)
hazard_table.sort_values(['game_player_clue_id', 'start'], inplace=True)

res_list = []
for i, group in hazard_table.groupby('game_player_clue_id'):
    res_list.append(process_adoption_group(group))
hazard_table = pd.concat(res_list)

hazard_table['treatment_n_exposures'] = hazard_table['treatment']*hazard_table['n_exposures']
hazard_table['treatment_references'] = hazard_table['treatment']*hazard_table['references']
hazard_table['treatment_pl2'] = hazard_table['treatment']*hazard_table['pl2']
hazard_table['treatment_n_existing_beliefs'] = hazard_table['treatment']*hazard_table['n_existing_beliefs']
hazard_table['treatment_n_fresh_candidates'] = hazard_table['treatment']*hazard_table['n_fresh_candidates']
hazard_table['treatment_max_similarity_to_exposer'] = hazard_table['treatment']*hazard_table['max_similarity_to_exposer']
hazard_table['treatment_in_deads'] = hazard_table['treatment']*hazard_table['in_deads']*1.0

# only look at 
adoption_hazard_table = hazard_table[(hazard_table['have_belief']==0) & (hazard_table['n_exposures'] > 0)]
adoption_hazard_table['adopt_event'] *=1
adoption_hazard_table['forget_event'] *=1
adoption_hazard_table.reset_index(drop=True, inplace=True)

hazard_table_collector.append(adoption_hazard_table)


#datafilename = "adoption_hazard_table.csv"
#adoption_hazard_table.to_csv(datafilename)

# def who_has_what(g):
#     holds = []
#     for n in g:
#         pos = g.node[n]['pos']
#         for cl in t_spokes+c_spokes:
#             edge = game['data.clues'][cl]['nodes']
#             holds.append((pos, cl, g.node[n]['M'].has_edge(*edge)))

#     holds_df = pd.DataFrame(holds, columns=['pos', 'clueId', 'present']).pivot(index='pos', columns='clueId', values='present')
#     #holds_df.reset_index(inplace=True, drop=True)
#     return holds_df    
    
# collector = []
# for (player_id, g, t) in retrace(game):
#     adoptions = who_has_what(g)
#     pca.fit(adoptions.loc[t_pos, t_spokes])
#     res = {'t':t/60,
#            'treatment': 1,
#            'pc1':pca.explained_variance_ratio_[0],
#            'game': 1211
#           }  
#     collector.append(res)

#     pca.fit(adoptions.loc[c_pos, c_spokes])
#     res = {'t': t/60,
#            'treatment': 0,
#            'pc1': pca.explained_variance_ratio_[0],
#            'game': 1211
#           }  
#     collector.append(res)



0 2019-11-27T18:20:38.972Z
1 2019-12-11T17:08:10.631Z
2 2019-12-11T18:21:08.701Z


AssertionError: 

# Assemble all results

In [45]:
res_df = pd.DataFrame(results)
res_df['d5th'] = res_df['t_spoke_5th_similarity'] - res_df['c_spoke_5th_similarity']
res_df['d95th'] = res_df['t_spoke_95th_similarity'] - res_df['c_spoke_95th_similarity']
res_df['dpc1'] = res_df['t_spoke_pc1_variance'] - res_df['c_spoke_pc1_variance'] # delta btw control and treatment
res_df['dblpc1'] = res_df['c_spoke_pc1_variance'] - res_df['t_spoke_pc1_initial_variance'] # growth of control above baseline
res_df['pctpc1'] = res_df['dpc1']/res_df['dblpc1']
res_df

,game,t_spoke_pc1_initial_variance,c_spoke_pc1_variance,t_spoke_pc1_variance,c_spoke_5th_similarity,c_spoke_95th_similarity,t_spoke_5th_similarity,t_spoke_95th_similarity,d5th,d95th,dpc1,dblpc1,pctpc1
0,1001,0.184596,0.293609,0.323265,-0.239046,0.625751,-0.244894,0.692255,-0.005849,0.066504,0.029656,0.109013,0.272044
1,1127,0.143541,0.279501,0.263857,-0.198557,0.684319,-0.274634,0.628958,-0.076077,-0.055362,-0.015644,0.135961,-0.115064
2,1211,0.139744,0.259976,0.313525,-0.216591,0.735980,-0.195694,0.623932,0.020897,-0.112048,0.053549,0.120232,0.445382


In [46]:
res_df.mean()

game                            3.337042e+10
t_spoke_pc1_initial_variance    1.559604e-01
c_spoke_pc1_variance            2.776954e-01
t_spoke_pc1_variance            3.002157e-01
c_spoke_5th_similarity         -2.180648e-01
c_spoke_95th_similarity         6.820169e-01
t_spoke_5th_similarity         -2.384076e-01
t_spoke_95th_similarity         6.483814e-01
d5th                           -2.034282e-02
d95th                          -3.363550e-02
dpc1                            2.252035e-02
dblpc1                          1.217350e-01
pctpc1                          2.007873e-01
dtype: float64

In [33]:
res_df.std()

c_spoke_pc1_variance       0.016889
t_spoke_pc1_variance       0.031862
c_spoke_5th_similarity     0.020284
c_spoke_95th_similarity    0.055150
t_spoke_5th_similarity     0.039868
t_spoke_95th_similarity    0.038079
d5th                       0.050086
d95th                      0.091237
dpc1                       0.035144
dtype: float64

In [83]:
(res_df['t_spoke_pc1_variance'] - res_df['c_spoke_pc1_variance'])/((res_df['t_spoke_pc1_variance'] + res_df['c_spoke_pc1_variance'])/2)

0    0.082030
1    0.281271
2    0.096150
3   -0.057583
dtype: float64

In [35]:
results

[{'game': '1127',
  't_spoke_pc1_initial_variance': array([0.14354067]),
  'c_spoke_pc1_variance': 0.27950137564179695,
  't_spoke_pc1_variance': 0.2638571946633557,
  'c_spoke_5th_similarity': -0.19855718635177688,
  'c_spoke_95th_similarity': 0.684319353018565,
  't_spoke_5th_similarity': -0.2746344822753012,
  't_spoke_95th_similarity': 0.6289576373413788},
 {'game': '1211',
  't_spoke_pc1_initial_variance': array([0.1845962]),
  'c_spoke_pc1_variance': 0.2599759304969385,
  't_spoke_pc1_variance': 0.31352497343973956,
  'c_spoke_5th_similarity': -0.21659149724763585,
  'c_spoke_95th_similarity': 0.7359800721939875,
  't_spoke_5th_similarity': -0.19569405342155582,
  't_spoke_95th_similarity': 0.623931623931624},
 {'game': '1001',
  'c_spoke_pc1_variance': 0.2936087578985613,
  't_spoke_pc1_variance': 0.32326493401139283,
  'c_spoke_5th_similarity': -0.23904572186687872,
  'c_spoke_95th_similarity': 0.6257513291440165,
  't_spoke_5th_similarity': -0.2448943327776391,
  't_spoke_95th

In [21]:
res

{'game': '1211',
 't_spoke_pc1_initial_variance': array([0.13974429]),
 'c_spoke_pc1_variance': 0.2599759304969385,
 't_spoke_pc1_variance': 0.31352497343973956,
 'c_spoke_5th_similarity': -0.21659149724763585,
 'c_spoke_95th_similarity': 0.7359800721939875,
 't_spoke_5th_similarity': -0.19569405342155582,
 't_spoke_95th_similarity': 0.623931623931624}